# Calculate area of subwatersheds

Substituting the ArcPy library within Working_ML_McAvoy_EricEdits notebook
with open source libraries(example: GDAL, Fiona, Shapely and GeoPandas)

In [1]:
from osgeo import ogr
from osgeo import osr
import geopandas as gpd
import fiona
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import os
from time import time

In [3]:
st = time()

# Set Local Variables
output_workspace = r'P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\working_edits_test'

#arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####
# Read any shapefile with geopandas

#set source for shapefile with all 11 subwatersheds in the county
all_subwatersheds_gpd = gpd.read_file(r"P:\Temp\Bakinam_Essawy\ML_Project\shapefiles\single_huc_test.shp")

#set source for streams data
streams_gpd = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\stream data\MarionCounty_Streams_Edited.shp")


#####these are the same for all subwatersheds in Indiana#####


#set source for dem
dem_Indiana = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\Indiana_3dep\Indiana_dem2"

#set source for slope dem
dem_slope = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\Indiana_SlopeRaster\IN_slope2"


#set source for NHFL Data
#nfhl_sfha = r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\NFHL_18_20200310.gdb\S_FLD_HAZ_AR"
nfhl_sfha = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\NFHL_18_20200310.gdb", driver='FileGDB', layer='S_FLD_HAZ_AR')

#set source for water bodies data 
water_bodies = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019"

#set source for railraods data
railroads = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\MarionCounty_Boundary\Marion_County_Boundary.shp")
  
#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp"

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\MarionCounty_Rainfall\All_Rainfall_Clipped_IN"

#set source for nlcd land use data
lu_usa = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\NLCD_Impervious\NLCD_2016_Impervious_L48_20190405_PERCENT\NLCD_2016_Impervious_L48_20190405.img"

#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

5.45 minutes to process.


In [ ]:
print('The spatial reference is', all_subwatersheds.crs)

In [4]:
## Empty lists

subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []

In [13]:
# Check the spatial reference for the geopandas
print('The spatial reference is', all_subwatersheds_gpd.crs)
all_subwatersheds_gpd.head(2)

The spatial reference is epsg:4269


,TNMID,GNIS_ID,AREAACRES,AREASQKM,STATES,HUC12,NAME,TOHUC,Perimeter,area_full,AREA,geometry
0,{A6F5714B-172F-424D-91F9-9345D6BC9D9A},0,17492.68,70.79,IN,051202011109,Little Eagle Creek,051202011110,55.883574,70.790345,70.790345,"POLYGON ((-86.21579 39.80945, -86.21506 39.809..."


In [8]:
# Convert geopandas projection to ESRI: 102008 so area and length is calculated in meters
all_subwatersheds = all_subwatersheds_gpd.copy()
all_subwatersheds= all_subwatersheds.to_crs('esri:102008')
print(all_subwatersheds.crs)


esri:102008


In [11]:
# Add area and perimeter fields for the whole subwatersheds file
#calculate area and perimeter of subwatersheds
all_subwatersheds["perimeter_geopandas"] = all_subwatersheds['geometry'].length/1000
all_subwatersheds["area_geopandas"] = all_subwatersheds['geometry'].area/10**6
all_subwatersheds.head(2)

,TNMID,GNIS_ID,AREAACRES,AREASQKM,STATES,HUC12,NAME,TOHUC,Perimeter,area_full,AREA,geometry,perimeter_geopandas,area_geopandas
0,{A6F5714B-172F-424D-91F9-9345D6BC9D9A},0,17492.68,70.79,IN,051202011109,Little Eagle Creek,051202011110,55.883574,70.790345,70.790345,"POLYGON ((787369.681 18192.954, 787421.550 182...",56.409334,70.79033


In [36]:
# Read a feature layer from Geodatabase using Geopandas
nfhl_sfha = gpd.read_file(r"P:\Temp\McAvoy\ML_DataCollection\MeridianHills\MeridianHills_DataCollection\NFHL_18_20200310.gdb", driver='FileGDB', layer='S_FLD_HAZ_AR')

In [44]:
nfhl_sfha.head(1)

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,...,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,SHAPE_Length,SHAPE_Area,geometry
0,18097C,1.1.1.0,18097C_1,NP,AH,None,T,740.0,NAVD88,-9999.0,...,,,-9999.0,-9999.0,,18097C_LOMC19,87bee992-4fa3-4527-bd71-935329abcbd8,0.002667,4.348833e-07,"MULTIPOLYGON (((-86.22136 39.81742, -86.22134 ..."


In [42]:
nfhl_sfha.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [27]:
all_subwatersheds_trans = all_subwatersheds.to_crs('esri:102008')

In [30]:
es_union = gpd.overlay(all_subwatersheds_trans, nfhl_sfha_gpd, how='union')

In [31]:
es_union

,TNMID_1,GNIS_ID_1,AREAACRES_1,AREASQKM_1,STATES_1,HUC12_1,NAME_1,TOHUC_1,Perimeter_1,area_full_1,...,AREAACRES_2,AREASQKM_2,STATES_2,HUC12_2,NAME_2,TOHUC_2,Perimeter_2,area_full_2,AREA_2,geometry
0,{A6F5714B-172F-424D-91F9-9345D6BC9D9A},0,17492.68,70.79,IN,051202011109,Little Eagle Creek,051202011110,55.883574,70.790345,...,17492.68,70.79,IN,051202011109,Little Eagle Creek,051202011110,55.883574,70.790345,70.790345,"POLYGON ((787369.681 18192.954, 787421.550 182..."
